In [1]:
import sys
sys.path.append(r'C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio/Classification/dsa_single_model')
import json
import pandas as pd
import yaml
import pickle
import os
from utils.feat_eng_pipeline import feat_eng_pipeline
from sklearn.model_selection import train_test_split


# Carregando as configurações do arquivo YAML
yaml_path = r"C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio\Classification\dsa_single_model\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

params = {
        'input_data':os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['input']),                
        'output_x_train' : os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['X_train']),        
        'output_x_val' : os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['X_val']),        
        'output_y_train' : os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['y_train']),        
        'output_y_val' : os.path.join(
            config['feat_selection']['path'],
            config['feat_selection']['y_val']),       
        'random_state' : config['feat_selection_params']['random_state'],
        'val_size' : config['feat_selection_params']['val_size'],
        'cols_2_drop' : config['feat_selection_params']['cols_2_drop'],
        'num_var' : config['feat_selection_params']['num_var'],
        'num_var_1' : config['feat_selection_params']['num_var_1'],
        'num_var_2' : config['feat_selection_params']['num_var_2'],
        'cat_var' : config['feat_selection_params']['cat_var'],
        'target' : config['feat_selection_params']['target'],
        'pipe': config['pipe_feat_eng']['path'], 
        'reports': config['save_reports']['path_reports'],
        'pipe_version': config['feat_selection_params']['pipe_version']        
        }
    

C:\Users\gustavo\Documents\Data Science\08-GitHub\Portifolio/Classification/dsa_single_model\utils\feat_eng_pipeline.py:40: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ("inputer_pipe", inputer_pipe, num_var_1)


In [2]:
df = pd.read_parquet(params['input_data'])
df.drop(
        columns=params['cols_2_drop'], 
        inplace=True)

In [3]:
X_train, X_val, y_train, y_val =  train_test_split(
        df.drop(columns=params['target']), 
        df[params['target']],
        test_size=params['val_size'], 
        random_state=params['random_state'])

In [5]:
pipe

NameError: name 'pipe' is not defined

In [ ]:
pipe = feat_eng_pipeline(
        num_var_1=params['num_var_1'],
        num_var_2=params['num_var_2'],
        )

TypeError: 'tuple' object is not callable

In [ ]:
 
# print('Feature Eng pipe transform')
# pipe.fit(X_train, y_train)

In [ ]:
# X_train_trans = pipe.transform(X_train)
# X_val_trans = pipe.transform(X_val)

In [ ]:
# X_train_trans

In [6]:
num_var_1=params['num_var_1']
num_var_2=params['num_var_2']

In [ ]:
from sklearn.pipeline import make_pipeline
from feature_engine.imputation import MeanMedianImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from feature_engine.outliers import Winsorizer
from feature_engine.transformation import LogCpTransformer
from sklearn.preprocessing import MinMaxScaler
from feature_engine.discretisation import DecisionTreeDiscretiser
from utils.functions import MedianByYTransformer
from feature_engine.scaling import MeanNormalizationScaler
from sklearn.impute import KNNImputer


   
    # numerical var
median = make_pipeline(
        MeanMedianImputer(
        imputation_method = 'median',
        variables = num_var_2))
    
inputer = KNNImputer()
inputer_pipe = make_pipeline(inputer)
   

preprocessor_1  = ColumnTransformer(
    transformers = [
        ("inputer_pipe", inputer_pipe, num_var_1),
        ("numerical_pipe", median, num_var_2)
        ]
    )  
    
    
pipe = make_pipeline(
         preprocessor_1.set_output(transform="pandas"),
 
        
        )
    


In [12]:
pipe.fit_transform(X_train, y_train)

,inputer_pipe__insulina,inputer_pipe__glicose,inputer_pipe__grossura_pele,inputer_pipe__bmi,numerical_pipe__num_gestacoes,numerical_pipe__pressao_sanguinea,numerical_pipe__indice_historico,numerical_pipe__idade
108,18.0,83.0,31.0,34.3,3,58,0.336,25
352,49.0,61.0,28.0,34.4,3,82,0.243,46
238,180.2,164.0,21.0,30.8,9,84,0.831,32
298,184.0,100.0,25.0,36.6,14,78,0.412,46
300,123.6,167.0,33.8,32.3,0,0,0.839,30
...,...,...,...,...,...,...,...,...
237,252.0,179.0,27.0,44.1,0,90,0.686,23
31,245.0,158.0,36.0,31.6,3,76,0.851,28
488,74.2,99.0,17.0,25.6,4,72,0.294,28
40,70.0,180.0,25.0,34.0,3,64,0.271,26
